# Sqoop Command

command:
sqoop --options-file /root/sq_options.txt


contents of sq_options.txt:


In [ ]:
import

--connect
jdbc:sqlserver://bigdata220w18.database.windows.net:1433;database=week3

--username
bigdata

--password
5Zgv\6;8rM

--compress

--compression-codec
snappy 

--as-parquetfile 

--target-dir 
/sqoop/wk3_hw_denormal_data 

--num-mappers
1

--query
select \
	transaction_data.household_key, \
	homeowner_desc, \
	day, \
	basket_id, \
	sales_value \  
from hh_demographic \
join transaction_data \
	on transaction_data.household_key = hh_demographic.household_key \
join product \
	on product.product_id = transaction_data.product_id \
WHERE $CONDITIONS

# Load Parquet File

In [1]:
df = sqlContext.read.parquet("hdfs://sandbox.hortonworks.com:8020/sqoop/wk3_hw_denormal_data/*")
df.show()

+-------------+--------------+---+-----------+-----------+
|household_key|homeowner_desc|day|  basket_id|sales_value|
+-------------+--------------+---+-----------+-----------+
|         1364|     Homeowner|  1|26984896261|       2.19|
|         1364|     Homeowner|  1|26984896261|       2.99|
|         1364|     Homeowner|  1|26984896261|       3.09|
|         1364|     Homeowner|  1|26984896261|        2.5|
|         1364|     Homeowner|  1|26984896261|        0.6|
|         1130|        Renter|  1|26984905972|       0.34|
|         1130|        Renter|  1|26984905972|       0.34|
|         1130|        Renter|  1|26984905972|       0.34|
|         1130|        Renter|  1|26984905972|       1.19|
|         1130|        Renter|  1|26984905972|       0.34|
|           98|       Unknown|  1|26984951769|        0.9|
|           98|       Unknown|  1|26984951769|        3.0|
|           98|       Unknown|  1|26984951769|       1.25|
|           98|       Unknown|  1|26984951769|       0.3

# Run Queries

## All RFM Metrics


In [7]:
df.registerTempTable("household_transaction_product")

sql_stmt = """
select 
  household_key, 
  max(day) as recency,
  count(distinct(basket_id)) as frequency,
  round(sum(sales_value), 2) as monetary
from household_transaction_product
group by household_key
order by recency desc, frequency desc, monetary desc
"""

rfm_metric_df = sqlContext.sql(sql_stmt)
rfm_metric_df.show(20)

rfm_metric_df.write.csv('hdfs://sandbox.hortonworks.com:8020/course2/rfm_metrics.csv')


+-------------+-------+---------+--------+
|household_key|recency|frequency|monetary|
+-------------+-------+---------+--------+
|          900|    711|     1223|16450.53|
|          762|    711|      886| 8982.79|
|         1901|    711|      852|11462.24|
|         1467|    711|      835| 2957.82|
|         1228|    711|      797|14503.89|
|          956|    711|      621|10231.93|
|         1419|    711|      524| 7634.57|
|          707|    711|      498|19194.42|
|          771|    711|      497| 7990.97|
|          631|    711|      483| 9277.72|
|          334|    711|      466|  4827.8|
|         1753|    711|      456| 2964.05|
|          800|    711|      443|10426.58|
|           19|    711|      421|11299.61|
|         1609|    711|      412|27859.68|
|         1166|    711|      409|12300.62|
|           27|    711|      405| 5736.24|
|         2194|    711|      404| 7169.78|
|         2198|    711|      403| 7205.88|
|         2400|    711|      391|14681.32|
+----------

## Homeowners RFM Metrics

In [8]:
sql_stmt = """
select 
  household_key, 
  max(day) as recency,
  count(distinct(basket_id)) as frequency,
  round(sum(sales_value), 2) as monetary
from household_transaction_product
where lower(homeowner_desc) = 'homeowner'
group by household_key
order by recency desc, frequency desc, monetary desc
"""

rfm_metric_df = sqlContext.sql(sql_stmt)
rfm_metric_df.show(20)

rfm_metric_df.write.csv('hdfs://sandbox.hortonworks.com:8020/course2/rfm_metrics_homeowners.csv')

+-------------+-------+---------+--------+
|household_key|recency|frequency|monetary|
+-------------+-------+---------+--------+
|          900|    711|     1223|16450.53|
|         1901|    711|      852|11462.24|
|          956|    711|      621|10231.93|
|          707|    711|      498|19194.42|
|          771|    711|      497| 7990.97|
|          631|    711|      483| 9277.72|
|          334|    711|      466|  4827.8|
|         1753|    711|      456| 2964.05|
|          800|    711|      443|10426.58|
|         1609|    711|      412|27859.68|
|         1166|    711|      409|12300.62|
|         2194|    711|      404| 7169.78|
|         2400|    711|      391|14681.32|
|         1229|    711|      385|18304.31|
|          906|    711|      382| 7048.71|
|         1823|    711|      377|12268.85|
|          358|    711|      371|12729.34|
|         2334|    711|      361|11955.38|
|         2085|    711|      359|12894.57|
|         2252|    711|      346| 9489.82|
+----------